In [11]:
import matplotlib.pyplot as plt
import json
import numpy as np 
import open3d as o3d
import mayavi.mlab
import sklearn.cluster
from sklearn import metrics

file_data = np.fromfile('/home/jidan/Documents/me5413/1_lidar/lidar_data/frame2.pcd.bin', dtype=np.float32)
pcd = o3d.io.read_point_cloud('/home/jidan/Documents/me5413/1_lidar/lidar_data/frame2.pcd.bin')
print(np.shape(pcd))
points = file_data.reshape((-1, 5))[:, :4]
print(np.shape(points))
# Data : (x, y, z, intensity)

x = points[:, 0]  # x position of point
y = points[:, 1]  # y position of point
z = points[:, 2]  # z position of point
r = points[:, 3]  # reflectance value of point
d = np.sqrt(x ** 2 + y ** 2)  # Map Distance from sensor
degr = np.degrees(np.arctan(z / d))
 
vals = 'height'
if vals == "height":
    col = z
else:
    col = d

def clustering(points, method):
    if method == 'dbscan':
        ## Do Clustering 
        db = sklearn.cluster.DBSCAN(eps=1.5, min_samples=5).fit(points)
        labels_db = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
        n_noise_ = list(labels_db).count(-1)

        print("Estimated number of clusters: %d" % n_clusters_)
        print("Estimated number of noise points: %d" % n_noise_)
        return labels_db

    if method == 'kmeans':
        kmeans = sklearn.cluster.KMeans(n_clusters=70, random_state=0, n_init="auto").fit(points)
        labels_km = kmeans.labels_
        return labels_km

    if method == 'meanshift':
        # The following bandwidth can be automatically detected using
        bandwidth = sklearn.cluster.estimate_bandwidth(points, quantile=0.1, n_samples=500)

        ms = sklearn.cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
        ms.fit(points)
        labels_ms = ms.labels_
        cluster_centers = ms.cluster_centers_

        labels_unique = np.unique(labels_ms)
        n_clusters_ = len(labels_unique)

        print("number of estimated clusters : %d" % n_clusters_)
        return labels_ms

    if method == 'optics':
        clust = sklearn.cluster.OPTICS(min_samples=50, xi=0.05, min_cluster_size=0.05)

        # Run the fit
        clust.fit(points)
        labels_op = clust.labels_
        return labels_op

#Clustering
method = 'dbscan'  #Options: 'dbscan','kmeans','optics','meanshift'
labels = clustering(points, method)

#Visualization and data saving
# fig = mayavi.mlab.figure(bgcolor=(0, 0, 0), size=(640, 500))
# mayavi.mlab.points3d(x, y, z,
#                      labels,  # Values used for Color
#                      mode="point",
#                      colormap='spectral',  # 'bone', 'copper', 'gnuplot'
#                      # color=(0, 1, 0),   # Used a fixed (r,g,b) instead
#                      figure=fig,
#                      )
# mayavi.mlab.show()

def vis(data,label):
    '''
    :param data: n*3的矩阵
    :param label: n*1的矩阵
    :return: 可视化
    '''
    data=data[:,:3]
    labels=np.asarray(label)
    max_label=labels.max()

    # 颜色
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    pt1 = o3d.geometry.PointCloud()
    pt1.points = o3d.utility.Vector3dVector(data.reshape(-1, 3))
    pt1.colors=o3d.utility.Vector3dVector(colors[:, :3])

    o3d.visualization.draw_geometries([pt1],'part of cloud',width=500,height=500)

vis(points[:,0:3],labels)

#Data saving
points[:,3] = labels
points = points.tolist()
with open('lidar_clustering.json', 'w') as f:
    json.dump(points, f)

[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for /home/jidan/Documents/me5413/1_lidar/lidar_data/frame2.pcd.bin (format: auto).
()
(34720, 4)
Estimated number of clusters: 365
Estimated number of noise points: 3438
